In [1]:
!pip install ultralytics
!pip install segment-anything
!pip install opencv-python-headless
!pip install matplotlib
!pip install kagglehub
!pip install supervision
!git clone https://github.com/IDEA-Research/GroundingDINO.git
%cd GroundingDINO
!pip install -e .
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!wget https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth -P checkpoints

fatal: destination path 'GroundingDINO' already exists and is not an empty directory.
/content/GroundingDINO
Obtaining file:///content/GroundingDINO
  Preparing metadata (setup.py) ... done
  Attempting uninstall: groundingdino
    Found existing installation: groundingdino 0.1.0
    Uninstalling groundingdino-0.1.0:
      Successfully uninstalled groundingdino-0.1.0
  Running setup.py develop for groundingdino
Looking in indexes: https://download.pytorch.org/whl/cu118
--2024-11-21 01:30:31--  https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-production-release-asset-2e65be/611591640/f221e500-c2fc-4fd3-b84e-8ad92a6923f3?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Credential=releaseassetproduction%2F20241121%2Fus-ea

In [3]:
from google.colab import drive
drive.mount('/content/drive')
base_dir = "/content/drive/MyDrive/Big_data_CNN_201124"

import os
if not os.path.exists(base_dir):
    os.makedirs(base_dir)

print(f"Carpeta base configurada en: {base_dir}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Carpeta base configurada en: /content/drive/MyDrive/Big_data_CNN_201124


In [4]:
import kagglehub
from shutil import copytree

path = kagglehub.dataset_download("sshikamaru/car-object-detection")
target_dir = os.path.join(base_dir, "car-object-detection")
copytree(path, target_dir, dirs_exist_ok=True)
print(f"Dataset descargado en: {target_dir}")

Dataset descargado en: /content/drive/MyDrive/Big_data_CNN_201124/car-object-detection


In [5]:
# Redimensionar imágenes
import cv2
from tqdm import tqdm

folders = [
    "/content/drive/MyDrive/Big_data_CNN_201124/car-object-detection/data/training_images",
    "/content/drive/MyDrive/Big_data_CNN_201124/car-object-detection/data/testing_images",
]

output_base_folder = "/content/drive/MyDrive/Big_data_CNN_201124/resized_images"
for input_folder in folders:
    output_folder = os.path.join(output_base_folder, os.path.basename(input_folder))
    os.makedirs(output_folder, exist_ok=True)
    print(f"Procesando imágenes de: {input_folder}")
    for filename in tqdm(os.listdir(input_folder)):
        if filename.endswith((".jpg", ".png")):
            img = cv2.imread(os.path.join(input_folder, filename))
            resized = cv2.resize(img, (128, 128), interpolation=cv2.INTER_AREA)
            cv2.imwrite(os.path.join(output_folder, filename), resized)

print(f"Imágenes redimensionadas guardadas en: {output_base_folder}")

Procesando imágenes de: /content/drive/MyDrive/Big_data_CNN_201124/car-object-detection/data/training_images


100%|██████████| 1001/1001 [00:39<00:00, 25.16it/s]


Procesando imágenes de: /content/drive/MyDrive/Big_data_CNN_201124/car-object-detection/data/testing_images


100%|██████████| 175/175 [00:04<00:00, 36.15it/s]


Imágenes redimensionadas guardadas en: /content/drive/MyDrive/Big_data_CNN_201124/resized_images


In [6]:
# Cargar modelo GroundingDINO
from groundingdino.util.inference import load_model, load_image, predict, annotate

model_config_path = "/content/GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py"
model_checkpoint_path = "/content/GroundingDINO/checkpoints/groundingdino_swint_ogc.pth"

model = load_model(model_config_path, model_checkpoint_path)
print("Modelo Grounding DINO cargado correctamente")

final text_encoder_type: bert-base-uncased


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.


Modelo Grounding DINO cargado correctamente


In [7]:
# Predicción y anotación para visualización
import supervision as sv

In [12]:
import os
from PIL import Image
from tqdm import tqdm
import supervision as sv
from groundingdino.util.inference import load_model, load_image, predict, annotate

# Configuración del modelo
model_config_path = "/content/GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py"
model_checkpoint_path = "/content/GroundingDINO/checkpoints/groundingdino_swint_ogc.pth"
model = load_model(model_config_path, model_checkpoint_path).to("cpu")  # Forzar a CPU
print("Modelo Grounding DINO cargado correctamente")

# Parámetros de predicción
text = "car"  # Texto para detección
BOX_THRESHOLD = 0.26
TEXT_THRESHOLD = 0.30

# Directorios de entrada y salida
input_folder = "/content/drive/MyDrive/Big_data_CNN_201124/resized_images/training_images"
output_folder = "/content/drive/MyDrive/Big_data_CNN_201124/detection"
os.makedirs(output_folder, exist_ok=True)

# Obtener las primeras 100 imágenes
all_files = [f for f in os.listdir(input_folder) if f.endswith((".jpg", ".png"))]
first_100_files = all_files[:100]

# Procesar las primeras 100 imágenes
for filename in tqdm(first_100_files):
    if filename.endswith((".jpg", ".png")):
        image_path = os.path.join(input_folder, filename)

        try:
            # Cargar imagen y preprocesar
            image_source, image_tensor = load_image(image_path)

            # Realizar predicción
            boxes, logits, phrases = predict(
                model=model,
                image=image_tensor,
                caption=text,
                box_threshold=BOX_THRESHOLD,
                text_threshold=TEXT_THRESHOLD,
                device="cpu"  # Cambiar a 'cuda' si tienes GPU
            )

            # Anotar la imagen
            annotated_frame = annotate(
                image_source=image_source,
                boxes=boxes,
                logits=logits,
                phrases=phrases
            )

            # Guardar la imagen anotada
            output_image_path = os.path.join(output_folder, filename)
            annotated_pil = Image.fromarray(annotated_frame)
            annotated_pil.save(output_image_path)

        except Exception as e:
            print(f"Error procesando {filename}: {e}")

print(f"Imágenes anotadas guardadas en: {output_folder}")

final text_encoder_type: bert-base-uncased
Modelo Grounding DINO cargado correctamente


100%|██████████| 100/100 [39:30<00:00, 23.71s/it]

Imágenes anotadas guardadas en: /content/drive/MyDrive/Big_data_CNN_201124/detection
